In [51]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [52]:
url_inicial = 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-numeros-anteriores-anio-2015'
url_root = 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-numeros-anteriores-anio-2015'
r=requests.get(url_inicial)

In [53]:
soup = BeautifulSoup(r.text, 'html.parser')

In [54]:
box = soup.find('div', class_='sumarios decada-2015')
volumen=box.findAll('div', class_='sumario row')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-sumario-vol-29-num-4-S0187533716X00027',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-sumario-vol-29-num-3-S0187533715X00045',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-sumario-vol-29-num-2-S0187533715X00033',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-sumario-vol-29-num-1-S0187533715X00021']

In [55]:
def get_url_items(sopa, url): 
    box = soup.find('div', class_='sumarios decada-2015')
    volumen=box.findAll('div', class_='sumario row')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [56]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links)

Estoy en la pagina https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-numeros-anteriores-anio-2015


In [57]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

4

In [58]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [59]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Perinatología y Reproducción Humana'
    a='III. Medicina y Ciencias de la Salud'
    tem='Perinatología, Neonatología, Obstetricia, Reproducción Humana'
    url='http://www.elsevier.es/rprh'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('div', class_='item-preview').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('a', class_='cover').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='titulo').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [60]:
list_scraper=list_scraper[0:4]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3


In [61]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Manteniendo el rumbo de laRevista Perinatologí...,None,http://www.elsevier.es/rprh,https://www.elsevier.es/es-revista-perinatolog...
1,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Plan estratégico 2015-2018 de la Revista Perin...,None,http://www.elsevier.es/rprh,https://www.elsevier.es/es-revista-perinatolog...
2,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",El embarazo en adolescentes y la participación...,None,http://www.elsevier.es/rprh,https://www.elsevier.es/es-revista-perinatolog...
3,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Perinatología y Reproducción HumanaArturo Card...,None,http://www.elsevier.es/rprh,https://www.elsevier.es/es-revista-perinatolog...


In [62]:
df_catalogo.to_csv('Revista95.04.csv', index=False)